In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.alpha = nn.Parameter(torch.tensor([0.1], requires_grad=True).to("cuda"))
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True
        )[0]
        u_x, u_t = u_x_t[:, 0], u_x_t[:, 1]
        u_xx = torch.autograd.grad(u_x, g, torch.ones_like(u_x), create_graph=True)[0][
            :, 0
        ]
        residual = u_t - self.alpha * u_xx
        return self.loss(residual, fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}, alpha {self.alpha.item()}")

In [8]:
model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
print(model)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): Tanh()
    (14): Linear(in_features=20, out_features=20, bias=True)
    (15): Tanh()
    (16): Linear(in_features=20, out_features=20, bias=True)
    (17): Tanh()
    (18): Linear(in_features=20, out_features=20, bias=True)
    (19): Tanh()
    (20): Linear(in_features=20, out_features=1, bias=True)
  )
  (loss): MSELoss()
)


In [25]:
import torch
import numpy as np
def loadAndPrep(u):
    x = [i for i in range(201)]
    t = [i for i in range(1024)]
    x = torch.tensor(x, dtype=torch.float32)
    t = torch.tensor(t, dtype=torch.float32)
    u = torch.tensor(u, dtype=torch.float32)
    X, T = np.meshgrid(x, t)
    xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
    idx = np.random.choice(u.flatten().shape[0], 10000, replace=False)
    xtrain = xtrue[idx, :]
    utrain = u.flatten()[idx][:, None]
    device = torch.device("cuda")
    xtrain = torch.tensor(xtrain, dtype=torch.float32, device=device)
    xtrue = torch.tensor(xtrue, dtype=torch.float32, device=device)
    utrain = utrain.to(device)
    utrue = u.flatten()[:, None]
    return xtrain, xtrue, utrain, utrue

In [26]:
import gc
def trainAndLog(u):
    xtrain, _, utrain, _ = loadAndPrep(u)
    model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
    loss = model.train_model(xtrain, utrain, epochs=15000)
    alpha = model.alpha.item()
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return alpha, loss

In [27]:
import h5py
with h5py.File("/kaggle/input/heateq/data.h5", "r") as f:
    print(f["0"].keys())

<KeysViewHDF5 ['alpha', 'u', 'u_noisy']>


In [ ]:
import random
import json

random.seed(69)
d = {}
with h5py.File("/kaggle/input/heateq/data.h5", "r") as f:
    a = random.choices(list(f.keys()), k=50)
    n = 0
    for i in a:
        print(n)
        d[i] = []
        d[i].append({"alpha" : f[i]["alpha"][()]})
        uclean = f[i]["u"][:]
        pred, loss = trainAndLog(uclean)
        d[i].append({"clean" : [{"predicted" : pred, "loss" : loss}]})
        unoisy = f[i]["u_noisy"][:]
        pred, loss = trainAndLog(unoisy)
        d[i].append({"noisy": [{"predicted" : pred, "loss" : loss}]})
        n += 1
        with open("/kaggle/working/results_heat.json", "w") as g:
            json.dump(d, g)

0


  0%|          | 0/15000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:825: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([10000, 1])) that is different to the input size (torch.Size([10000])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 8/15000 [00:01<26:26,  9.45it/s]  

Epoch 0, Loss 0.02058250643312931, alpha 0.09999560564756393


  7%|▋         | 1015/15000 [00:15<03:14, 71.93it/s]

Epoch 1000, Loss 0.004113102797418833, alpha -0.3004153370857239


...

 60%|██████    | 9016/15000 [02:05<01:23, 71.66it/s]

Epoch 9000, Loss 0.11094319820404053, alpha -0.008793833665549755


 67%|██████▋   | 10016/15000 [02:19<01:09, 71.65it/s]

Epoch 10000, Loss 0.11083211749792099, alpha -0.011278800666332245


 72%|███████▏  | 10833/15000 [02:30<00:57, 71.89it/s]